In [ ]:
from flask import Flask, render_template
import sqlite3
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import logging

# Initialize Flask App
app = Flask(__name__)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_meteorite_data(db_file='nasa_data.db'):
    """Fetch meteorite data from the SQLite database."""
    try:
        conn = sqlite3.connect(db_file)
        query = '''
        SELECT name, year, "mass (g)", reclat, reclong
        FROM meteorite_landings
        WHERE reclat IS NOT NULL AND reclong IS NOT NULL AND "mass (g)" IS NOT NULL AND year IS NOT NULL
        '''
        df = pd.read_sql(query, conn)
        df.columns = ['name', 'year', 'mass (g)', 'reclat', 'reclong']
        conn.close()
        return df
    except Exception as e:
        logger.error(f"Error fetching data from database: {e}")
        raise

@app.route('/')
def index():
    """Homepage showing visualization links."""
    return render_template('index.html')

@app.route('/data')
def data():
    """Page displaying the data tables."""
    try:
        conn = sqlite3.connect('nasa_data.db')
        neows_data = pd.read_sql('SELECT * FROM neows_data', conn).to_html(classes='table', index=False)
        meteorite_data = pd.read_sql('SELECT * FROM meteorite_landings', conn).to_html(classes='table', index=False)
        conn.close()
        return render_template('data.html', neows_data=neows_data, meteorite_data=meteorite_data)
    except Exception as e:
        logger.error(f"Error fetching data tables: {e}")
        return str(e)

@app.route('/plot')
def plot():
    """Page displaying the Plotly globe visualization."""
    try:
        # Fetch data from the database
        data = fetch_meteorite_data()

        # Filter to avoid plotting too many points for performance, sample 2000 random points
        subset_data = data.sample(n=2000, random_state=1)

        # Assign a color based on the mass of the meteorite, using a logarithmic scale for better color distribution
        mass_log = np.log10(subset_data['mass (g)'] + 1)  # +1 to avoid log(0)
        colorscale = 'Viridis'  # Using a perceptually uniform colorscale

        # Create a 3D globe visualization using Plotly
        fig = go.Figure(data=go.Scattergeo(
            lon=subset_data['reclong'],
            lat=subset_data['reclat'],
            text=subset_data['name'] + " - Year: " + subset_data['year'].astype(str) + " - Mass(g): " + subset_data['mass (g)'].astype(str),
            mode='markers',
            marker=dict(
                size=4 + (mass_log * 2),  # Dynamic sizing based on mass
                color=mass_log,
                colorscale=colorscale,
                colorbar=dict(title='Log10(Mass)'),
                line_color='black',
                line_width=0.5
            )))

        fig.update_layout(
            title='Meteorite Landings Visualization on a 3D Globe',
            title_x=0.5,  # Center title
            width=1200,  # Increase the size of the globe in the plot
            height=800,
            geo=dict(
                projection_type='orthographic',
                showland=True,
                landcolor='rgb(243, 243, 243)',
                countrycolor='rgb(204, 204, 204)',
                coastlinecolor='rgb(106, 106, 106)',
                oceancolor='lightblue',
                projection_rotation=dict(lon=60, lat=0, roll=0),
            ),
            margin=dict(l=10, r=10, t=40, b=10)
        )

        # Save Plotly figure to an HTML file
        fig.write_html('templates/plotly_globe.html', full_html=False)

        return render_template('plotly_globe.html')
    except Exception as e:
        logger.error(f"Error generating Plotly globe: {e}")
        return str(e)

if __name__ == '__main__':
    try:
        app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)
    except Exception as e:
        logger.error(f"Error starting the server: {e}")
